In [29]:
import numpy as np
import pandas as pd
#for reading in data properly
import ast
import json

import gensim
from gensim.models import Doc2Vec
from gensim.models import Word2Vec
from gensim.models.doc2vec import TaggedDocument

from sklearn.model_selection import train_test_split
from sklearn import utils

import re

import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

stop_words = set(stopwords.words('english'))

In [30]:
all_data = pd.read_csv('train.csv')
all_data = all_data.dropna(subset=['overview', 'genres']) #drop cols without overview or genre (data we use or labels)
genre_set = {'Comedy'}

In [31]:
def text_to_list(x):
    if pd.isna(x):
        return ''
    else:
        return ast.literal_eval(x)

def parse_json(x):
    try:
        return json.loads(x.replace("'", '"'))[0]['name']
    except:
        return ''
    
def parse_all_genres_json(x):
    try:
        json_genres = json.loads(x.replace("'", '"'))
        numElems = len(json_genres)
        for i in range(numElems):
            genre_set.add(json_genres[i]['name'])
    except:
        return ''
    
def parse_genres_json(x):
    try:
        json_genres = json.loads(x.replace("'", '"'))
        numElems = len(json_genres)
        ret = [0]*len(genre_dict) #20 0s
        for i in range(numElems):
            ret[genre_dict[(json_genres[i]['name'])]] = 1
        return ret
    except:
        return ''
    

def get_labels_as_strs(x):
    try:
        json_genres = json.loads(x.replace("'", '"'))
        numElems = len(json_genres)
        ret = []#20 0s
        for i in range(numElems):
            ret.append(json_genres[i]['name'])
        return ret
    except:
        return ''

In [32]:
 def getAllGenres():
    full_data = pd.read_csv('train.csv')

    y = full_data['genres']
    y.apply(parse_all_genres_json)

In [33]:
getAllGenres()#populate the genre set

In [34]:
#get set to dictionary for indexing of target vectors
genre_dict = {}
index = 0
for genre in genre_set:
    genre_dict[genre] = index
    index += 1

In [35]:
def getGenresVects():
    y = all_data['genres']
    ret = y.apply(parse_genres_json)
    all_data['genres_vect'] = ret
    label_strs = y.apply(get_labels_as_strs)
    all_data['genres_labels'] = label_strs
    return ret

In [36]:
labels_vects = getGenresVects() #get label vectors for genres indexed by indexes in genre_dict

In [37]:
#put to lower case, remove punctation
def cleanText(text):
    text = re.sub(r'[^a-z A-Z0-9]', "", text) #maybe shouldn't remove punction between words here?
    text = text.lower()
    return text
all_data['cleanOverview'] = all_data['overview'].apply(cleanText)

In [38]:
#logistic regression data
lr_data = all_data[['cleanOverview', 'genres_labels', 'genres_vect', 'overview']]

In [39]:
train, test = train_test_split(lr_data, test_size=0.2, random_state=42)

CNN STUFF here

In [76]:
#get word embeddings
x = lr_data['cleanOverview'].values.tolist()
y = lr_data['genres_vect']

In [77]:
y_train = y.tolist()
y_train = np.array(y_train)

In [41]:
tok = [word_tokenize(sent) for sent in x]

In [42]:
model = Word2Vec(tok, min_count = 1, size=32)

In [43]:
model.wv.__getitem__('test')

array([ 0.05091951, -0.04784649,  0.11069489, -0.24993515,  0.20700265,
        0.06569795, -0.2701927 ,  0.41776818, -0.1219014 ,  0.42027968,
       -0.33430973,  0.01497537, -0.52383196,  0.08832876,  0.11108792,
       -0.01976492,  0.27744392,  0.4145325 ,  0.1559598 ,  0.56155735,
       -0.30898437, -0.00444493, -0.11159223,  0.3572997 , -0.14436567,
       -0.03715143, -0.06922381, -0.19020006, -0.31129828, -0.17555408,
       -0.5060845 , -0.565167  ], dtype=float32)

In [53]:
from keras.layers import Conv1D, GlobalMaxPooling1D
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.layers import Flatten
from keras.layers.embeddings import Embedding
structure_test = Sequential()

#below should not be real code, it is just to confirm our network structure is good

#32 because word vectors were made to have length of 32
#200 as largest overview has 177 words and we'd like to include enttire thing
e = Embedding(100000, 32, input_length=200)
structure_test.add(e)
structure_test.add(Conv1D(filters=100, kernel_size=2, padding='valid', activation='relu', strides=1))
structure_test.add(GlobalMaxPooling1D())
structure_test.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_4 (Embedding)      (None, 200, 32)           3200000   
_________________________________________________________________
conv1d_4 (Conv1D)            (None, 199, 100)          6500      
_________________________________________________________________
global_max_pooling1d_3 (Glob (None, 100)               0         
Total params: 3,206,500
Trainable params: 3,206,500
Non-trainable params: 0
_________________________________________________________________


In [135]:
model_cnn_01 = Sequential()
e = Embedding(100000, 32, weights=[embedding_matrix], input_length=200, trainable=False)
model_cnn_01.add(e)
model_cnn_01.add(Conv1D(filters=100, kernel_size=2, padding='valid', activation='relu', strides=1))
model_cnn_01.add(GlobalMaxPooling1D())
model_cnn_01.add(Dense(256, activation='relu'))
model_cnn_01.add(Dense(20, activation='sigmoid'))
model_cnn_01.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
#model_cnn_01.fit(x_train_seq, y_train, validation_data=(x_val_seq, y_validation), epochs=5, batch_size=32, verbose=2)
model_cnn_01.fit(x_train_seq, y_train, epochs=1000, batch_size=32, verbose=2)

Epoch 1/1000
 - 2s - loss: 0.3302 - acc: 0.8670
Epoch 2/1000
 - 2s - loss: 0.3143 - acc: 0.8749
Epoch 3/1000
 - 2s - loss: 0.3150 - acc: 0.8748
Epoch 4/1000
 - 2s - loss: 0.3144 - acc: 0.8749
Epoch 5/1000
 - 2s - loss: 0.3141 - acc: 0.8749
Epoch 6/1000
 - 2s - loss: 0.3138 - acc: 0.8755
Epoch 7/1000
 - 2s - loss: 0.3136 - acc: 0.8756
Epoch 8/1000
 - 2s - loss: 0.3136 - acc: 0.8761
Epoch 9/1000
 - 2s - loss: 0.3137 - acc: 0.8752
Epoch 10/1000
 - 2s - loss: 0.3130 - acc: 0.8749
Epoch 11/1000
 - 2s - loss: 0.3130 - acc: 0.8756
Epoch 12/1000
 - 2s - loss: 0.3137 - acc: 0.8754
Epoch 13/1000
 - 2s - loss: 0.3124 - acc: 0.8757
Epoch 14/1000
 - 2s - loss: 0.3127 - acc: 0.8762
Epoch 15/1000
 - 2s - loss: 0.3120 - acc: 0.8756
Epoch 16/1000
 - 2s - loss: 0.3115 - acc: 0.8763
Epoch 17/1000
 - 2s - loss: 0.3115 - acc: 0.8760
Epoch 18/1000
 - 2s - loss: 0.3109 - acc: 0.8761
Epoch 19/1000
 - 2s - loss: 0.3105 - acc: 0.8755
Epoch 20/1000
 - 2s - loss: 0.3109 - acc: 0.8758
Epoch 21/1000
 - 2s - loss: 0

 - 2s - loss: 0.2792 - acc: 0.8852
Epoch 168/1000
 - 2s - loss: 0.2790 - acc: 0.8855
Epoch 169/1000
 - 2s - loss: 0.2781 - acc: 0.8853
Epoch 170/1000
 - 2s - loss: 0.2787 - acc: 0.8857
Epoch 171/1000
 - 2s - loss: 0.2772 - acc: 0.8861
Epoch 172/1000
 - 2s - loss: 0.2778 - acc: 0.8858
Epoch 173/1000
 - 2s - loss: 0.2779 - acc: 0.8859
Epoch 174/1000
 - 2s - loss: 0.2773 - acc: 0.8862
Epoch 175/1000
 - 2s - loss: 0.2763 - acc: 0.8860
Epoch 176/1000
 - 2s - loss: 0.2766 - acc: 0.8860
Epoch 177/1000
 - 2s - loss: 0.2758 - acc: 0.8870
Epoch 178/1000
 - 2s - loss: 0.2759 - acc: 0.8866
Epoch 179/1000
 - 2s - loss: 0.2756 - acc: 0.8866
Epoch 180/1000
 - 2s - loss: 0.2750 - acc: 0.8869
Epoch 181/1000
 - 2s - loss: 0.2745 - acc: 0.8869
Epoch 182/1000
 - 2s - loss: 0.2741 - acc: 0.8865
Epoch 183/1000
 - 2s - loss: 0.2738 - acc: 0.8870
Epoch 184/1000
 - 2s - loss: 0.2731 - acc: 0.8865
Epoch 185/1000
 - 2s - loss: 0.2721 - acc: 0.8877
Epoch 186/1000
 - 2s - loss: 0.2723 - acc: 0.8882
Epoch 187/1000


 - 2s - loss: 0.2235 - acc: 0.9061
Epoch 332/1000
 - 2s - loss: 0.2237 - acc: 0.9067
Epoch 333/1000
 - 2s - loss: 0.2237 - acc: 0.9059
Epoch 334/1000
 - 2s - loss: 0.2230 - acc: 0.9068
Epoch 335/1000
 - 2s - loss: 0.2243 - acc: 0.9053
Epoch 336/1000
 - 2s - loss: 0.2216 - acc: 0.9066
Epoch 337/1000
 - 2s - loss: 0.2227 - acc: 0.9058
Epoch 338/1000
 - 2s - loss: 0.2218 - acc: 0.9066
Epoch 339/1000
 - 2s - loss: 0.2217 - acc: 0.9071
Epoch 340/1000
 - 2s - loss: 0.2209 - acc: 0.9072
Epoch 341/1000
 - 2s - loss: 0.2223 - acc: 0.9070
Epoch 342/1000
 - 2s - loss: 0.2204 - acc: 0.9069
Epoch 343/1000
 - 2s - loss: 0.2207 - acc: 0.9073
Epoch 344/1000
 - 2s - loss: 0.2215 - acc: 0.9067
Epoch 345/1000
 - 2s - loss: 0.2213 - acc: 0.9070
Epoch 346/1000
 - 2s - loss: 0.2195 - acc: 0.9074
Epoch 347/1000
 - 2s - loss: 0.2195 - acc: 0.9080
Epoch 348/1000
 - 2s - loss: 0.2199 - acc: 0.9077
Epoch 349/1000
 - 2s - loss: 0.2184 - acc: 0.9072
Epoch 350/1000
 - 2s - loss: 0.2179 - acc: 0.9084
Epoch 351/1000


 - 2s - loss: 0.1804 - acc: 0.9247
Epoch 496/1000
 - 2s - loss: 0.1811 - acc: 0.9248
Epoch 497/1000
 - 2s - loss: 0.1790 - acc: 0.9259
Epoch 498/1000
 - 2s - loss: 0.1789 - acc: 0.9257
Epoch 499/1000
 - 2s - loss: 0.1792 - acc: 0.9251
Epoch 500/1000
 - 2s - loss: 0.1784 - acc: 0.9251
Epoch 501/1000
 - 2s - loss: 0.1771 - acc: 0.9257
Epoch 502/1000
 - 2s - loss: 0.1786 - acc: 0.9259
Epoch 503/1000
 - 2s - loss: 0.1772 - acc: 0.9260
Epoch 504/1000
 - 2s - loss: 0.1782 - acc: 0.9256
Epoch 505/1000
 - 2s - loss: 0.1780 - acc: 0.9251
Epoch 506/1000
 - 2s - loss: 0.1772 - acc: 0.9252
Epoch 507/1000
 - 2s - loss: 0.1763 - acc: 0.9273
Epoch 508/1000
 - 2s - loss: 0.1749 - acc: 0.9266
Epoch 509/1000
 - 2s - loss: 0.1763 - acc: 0.9274
Epoch 510/1000
 - 2s - loss: 0.1756 - acc: 0.9273
Epoch 511/1000
 - 2s - loss: 0.1763 - acc: 0.9263
Epoch 512/1000
 - 2s - loss: 0.1761 - acc: 0.9271
Epoch 513/1000
 - 2s - loss: 0.1757 - acc: 0.9275
Epoch 514/1000
 - 2s - loss: 0.1760 - acc: 0.9263
Epoch 515/1000


 - 2s - loss: 0.1463 - acc: 0.9399
Epoch 660/1000
 - 2s - loss: 0.1437 - acc: 0.9424
Epoch 661/1000
 - 2s - loss: 0.1429 - acc: 0.9428
Epoch 662/1000
 - 2s - loss: 0.1436 - acc: 0.9412
Epoch 663/1000
 - 2s - loss: 0.1429 - acc: 0.9422
Epoch 664/1000
 - 2s - loss: 0.1447 - acc: 0.9401
Epoch 665/1000
 - 2s - loss: 0.1440 - acc: 0.9408
Epoch 666/1000
 - 2s - loss: 0.1426 - acc: 0.9419
Epoch 667/1000
 - 2s - loss: 0.1433 - acc: 0.9419
Epoch 668/1000
 - 2s - loss: 0.1419 - acc: 0.9427
Epoch 669/1000
 - 2s - loss: 0.1433 - acc: 0.9419
Epoch 670/1000
 - 2s - loss: 0.1420 - acc: 0.9421
Epoch 671/1000
 - 2s - loss: 0.1412 - acc: 0.9426
Epoch 672/1000
 - 2s - loss: 0.1421 - acc: 0.9419
Epoch 673/1000
 - 2s - loss: 0.1414 - acc: 0.9414
Epoch 674/1000
 - 2s - loss: 0.1415 - acc: 0.9419
Epoch 675/1000
 - 2s - loss: 0.1411 - acc: 0.9427
Epoch 676/1000
 - 2s - loss: 0.1407 - acc: 0.9430
Epoch 677/1000
 - 2s - loss: 0.1406 - acc: 0.9422
Epoch 678/1000
 - 2s - loss: 0.1402 - acc: 0.9434
Epoch 679/1000


 - 2s - loss: 0.1153 - acc: 0.9547
Epoch 824/1000
 - 2s - loss: 0.1161 - acc: 0.9537
Epoch 825/1000
 - 2s - loss: 0.1159 - acc: 0.9543
Epoch 826/1000
 - 2s - loss: 0.1147 - acc: 0.9551
Epoch 827/1000
 - 2s - loss: 0.1153 - acc: 0.9538
Epoch 828/1000
 - 2s - loss: 0.1151 - acc: 0.9545
Epoch 829/1000
 - 2s - loss: 0.1133 - acc: 0.9556
Epoch 830/1000
 - 2s - loss: 0.1140 - acc: 0.9552
Epoch 831/1000
 - 2s - loss: 0.1137 - acc: 0.9552
Epoch 832/1000
 - 2s - loss: 0.1133 - acc: 0.9556
Epoch 833/1000
 - 2s - loss: 0.1132 - acc: 0.9556
Epoch 834/1000
 - 2s - loss: 0.1148 - acc: 0.9551
Epoch 835/1000
 - 2s - loss: 0.1131 - acc: 0.9552
Epoch 836/1000
 - 2s - loss: 0.1130 - acc: 0.9551
Epoch 837/1000
 - 2s - loss: 0.1139 - acc: 0.9552
Epoch 838/1000
 - 2s - loss: 0.1123 - acc: 0.9570
Epoch 839/1000
 - 2s - loss: 0.1127 - acc: 0.9556
Epoch 840/1000
 - 2s - loss: 0.1134 - acc: 0.9558
Epoch 841/1000
 - 2s - loss: 0.1124 - acc: 0.9569
Epoch 842/1000
 - 2s - loss: 0.1116 - acc: 0.9562
Epoch 843/1000


 - 2s - loss: 0.0927 - acc: 0.9648
Epoch 988/1000
 - 2s - loss: 0.0923 - acc: 0.9647
Epoch 989/1000
 - 2s - loss: 0.0916 - acc: 0.9657
Epoch 990/1000
 - 2s - loss: 0.0923 - acc: 0.9656
Epoch 991/1000
 - 2s - loss: 0.0933 - acc: 0.9644
Epoch 992/1000
 - 2s - loss: 0.0918 - acc: 0.9654
Epoch 993/1000
 - 2s - loss: 0.0931 - acc: 0.9643
Epoch 994/1000
 - 2s - loss: 0.0909 - acc: 0.9654
Epoch 995/1000
 - 2s - loss: 0.0903 - acc: 0.9664
Epoch 996/1000
 - 2s - loss: 0.0919 - acc: 0.9650
Epoch 997/1000
 - 2s - loss: 0.0906 - acc: 0.9663
Epoch 998/1000
 - 2s - loss: 0.0911 - acc: 0.9656
Epoch 999/1000
 - 2s - loss: 0.0916 - acc: 0.9651
Epoch 1000/1000
 - 2s - loss: 0.0919 - acc: 0.9653


In [136]:
res = model_cnn_01.predict(x_train_seq)

In [137]:
label_predictions = []
for i in range(res.shape[0]):
    pred = [0]*20
    for j in range(res.shape[1]):
        if res[i][j] >= .5:
            pred[j] = 1
    label_predictions.append(pred)

In [181]:
y_train[:,19].sum()

141

In [179]:
label_predictions = np.array(label_predictions)
label_predictions[:,19].sum()

143

In [98]:
y_train[0]

array([0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])

In [88]:
x_train_seq[0].shape

(200,)

In [64]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

tokenizer = Tokenizer()
tokenizer.fit_on_texts(x)
sequences = tokenizer.texts_to_sequences(x)

x_train_seq = pad_sequences(sequences, maxlen=200)

In [70]:
embeddings_index = {}
for w in model.wv.vocab.keys():
    embeddings_index[w] = model.wv[w]


num_words = 100000
embedding_matrix = np.zeros((num_words, 32))
for word, i in tokenizer.word_index.items():
    if i >= num_words:
        continue
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector